In [14]:
using CairoMakie
using CSV
using DataFrames
using MLJ
using MLJOpenML

In [12]:
p = DataFrame(load(42585))
describe(p)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Nothing,SubStrin…,Nothing,SubStrin…,Int64,DataType
1,species,,"""Adelie""",,"""Gentoo""",0,SubString{String}
2,island,,"""Biscoe""",,"""Torgersen""",0,SubString{String}
3,culmen_length_mm,,32.1,,?,0,SubString{String}
4,culmen_depth_mm,,13.1,,?,0,SubString{String}
5,flipper_length_mm,,172.0,,?,0,SubString{String}
6,body_mass_g,,2700.0,,?,0,SubString{String}
7,sex,,"""FEMALE""",,?,0,SubString{String}


In [15]:
# Import and unpack training dataset function
function get_data(csv_path, col_names, new_col_names, response_var)
    df = CSV.read(csv_path, DataFrame, missingstring="NA") |>
    x -> select(x, col_names) |>
    x -> rename(x, new_col_names) |>
    dropmissing |>
    x -> coerce(x, :pl=>Continuous, :sx=>Continuous, :snr=>Multiclass, :smr=>Multiclass)
    # Split predictors/responde variable
    y, X = unpack(df, ==(Symbol(response_var)), _->true; rng=1010)
    return y, X
end


get_data (generic function with 1 method)

In [16]:
csv_path = "C:/Users/julio/Downloads/lichen_training.csv"
col_names = ["Total.lichen", "basal_area", "PL", "mean_Long", "mean_Lat", "Aspect_south", "SNR", "SMR", "SX"]
new_col_names = ["biomass", "basalarea", "pl", "lon", "lat", "south", "snr", "smr", "sx"]
response_var = "biomass"

y, X = get_data(csv_path, col_names, new_col_names, response_var);

In [17]:
[col for col in names(X) if col.scitype <: Continuous]
# ohe = OneHotEncoder()

ErrorException: type String has no field scitype

In [27]:
# Create a train a model
function model_data(X, y)
    # Load model and create pipeline
    model = @load RandomForestRegressor pkg=DecisionTree verbosity=0
    continuous_cols = [col for names(X) if col.scitype <: Continuous]
    data_std = Standardizer(X)
    pipe = @pipeline Standardizer OneHotEncoder model target=v->log.(v) inverse=v->exp.(v)
    # Create a machine
    pipe.random_forest_regressor.n_trees = 50
    m = machine(pipe, X, y)

    return m
end

LoadError: LoadError: UndefVarError: model not defined
in expression starting at c:\Users\julio\Desktop\jl-sandbox\code\lichen-functions-linear-randomforest.ipynb:5

In [ ]:
# Split predictors (X) and response variable (y)
y, X = unpack(lichen_training, ==(:biomass_log), _ -> true);

In [ ]:
# Load a linear regressor and a random forest regressor
LinearRegressor = @load LinearRegressor pkg=MLJLinearModels
RFRegressor = @load RandomForestRegressor pkg=DecisionTree

In [ ]:
# Instantiate a pipeline
RegressorPipe = @pipeline(Standardizer(), LinearRegressor(), RFRegressor())